In [ ]:
#region imports
from AlgorithmImports import *
#endregion
from datetime import timedelta

class BasicTemplateOptionsAlgorithm(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2014, 1, 1)
        self.SetEndDate(2020, 1, 1)
        self.SetCash(100000)
        equity = self.AddEquity("SPY", Resolution.Minute)
        option = self.AddOption("SPY", Resolution.Minute)
        self.symbol = option.Symbol
        # set our strike/expiry filter for this option schain
        option.SetFilter(-5, 5, timedelta(0), timedelta(5))
        # use the underlying equity GOOG as the benchmark
        self.SetBenchmark(equity.Symbol)

    def OnData(self,slice):
        optionChain = slice.OptionChains
        for i in optionChain:
            if i.Key != self.symbol: continue
            chains = i.Value
            if not self.Portfolio.Invested:
                self.TradeOptions(chains)
            

    def TradeOptions(self, chain):
        # sort the optionchain by expiration date and choose the furthest date
        expiry = sorted(chain, key = lambda x: x.Expiry, reverse=True)[0].Expiry

        # filter the call options from the contracts expires on that date
        call = [i for i in chain if i.Expiry == expiry and i.Right == 0]
        put = [i for i in chain if i.Expiry == expiry and i.Right == 1]
        # sort the contracts according to their strike prices
        call_contracts = sorted(call, key = lambda x: x.Strike)
        put_contracts = sorted(call, key= lambda x: x.Strike, reverse=True)
        if len(call_contracts) == 0 or len(put_contracts) == 0: return

        self.call = call_contracts[0]
        for put_order in put_contracts:
            if put_order.Strike == self.call.Strike:
                self.put = put_order
                self.MarketOrder(self.call.Symbol, -1)
                self.MarketOrder(self.put.Symbol, -1)
                self.MarketOnCloseOrder(self.call.Symbol, 1)
                self.MarketOnCloseOrder(self.put.Symbol, 1)
                return

    def OnOrderEvent(self, orderEvent):
        self.Log(str(orderEvent))
    
    def OnEndOfDay(self, symbol):
        holdings = self.Portfolio[symbol]
        if holdings.Invested:
            if holdings.IsLong:
                self.Sell(symbol, holdings.Quantity)
            else:
                self.Buy(symbol, holdings.Quantity)